# Task #1 

A template code for training an RBM on H$_2$ data for $r = 1.2$ is shown here. Modify this!

Imports and loading in data:


## Run the notebook on google colab
To run the notebook on google colab press the badge below 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/olgOk/CohortProject_2020/blob/master/Project_1_RBM_and_Tomography/Task1_Oscar_JH.ipynb)

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import os

# Added code to run on google colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  ! git clone https://github.com/olgOk/CohortProject_2020.git
  from google.colab import drive
  drive.mount('/content/gdrive') 
  %cd Project_1_RBM_and_Tomography/
  ! git pull

from RBM_helper import RBM
import H2_energy_calculator

# # training_data = torch.from_numpy(np.loadtxt("H2_data/R_1.2_samples.txt"))
# training_data = torch.from_numpy(np.loadtxt("H2_data/R_1.2_samples.txt"))
coeff = np.loadtxt("H2_data/H2_coefficients.txt")[20,:]

# true_energy = H2_energy_calculator.energy_from_freq(training_data, coeff)
# print("H2 energy for r = 1.2: ",true_energy)
# print(type(training_data))



Define the RBM:

In [2]:
n_vis = 2
n_hin = 10

rbm = RBM(n_vis, n_hin)

Train the RBM:

In [92]:
def train_RBM( training_data): 
    
    epochs = 500 # number of training steps
    num_samples = 1000 # number of samples to generate from the RBM to calculate the H2 energy
    N_draws = 1000 # For quantifying the variance of the final model

    true_energy = H2_energy_calculator.energy_from_freq(training_data, coeff)
    #print("True energy: ",true_energy)

    for e in range(1, epochs+1):
        # do one epoch of training
        rbm.train(training_data)   

    # Draw several samples for the purpose of uncertainty quantification of the final results
    
    E_samples = np.zeros(N_draws)
    
    for k in range(0,N_draws):
        init_state = torch.zeros(num_samples, n_vis)
        RBM_samples = rbm.draw_samples(15, init_state)
        energies = H2_energy_calculator.energy(RBM_samples, coeff, rbm.wavefunction)
        E_samples[k] = energies.item()

    return E_samples,true_energy


def confidence_interval(x):
    '''
    Compute the 68% and 96% confidence inteval
    '''
    x_68 = np.percentile(x, [16,50,84])
    x_96 = np.percentile(x, [2,50,98])
    
    return x_68,x_96

In [93]:
# Sort the available radii first

radii = []

for entry in os.scandir('H2_data/'):
    if (entry.path.endswith("samples.txt")):
        radius =  float(entry.name.split("_")[1])
        radii.append(radius)

radii = np.array(radii)
radii.sort()

In [94]:
r_subset = []
E_true_r = []
E_rbm_r_med = []
E_rbm_r_low_68 = []
E_rbm_r_high_68 = []
E_rbm_r_low_96 = []
E_rbm_r_high_96 = []


for k in range(0,len(radii),5):
    rk = radii[k]
    
    r_subset.append(rk)
    
    file_name = 'H2_data/R_'+str(rk)+'_samples.txt'
    training_data = torch.from_numpy(np.loadtxt(file_name))
    E_rbm_samples, E_true = train_RBM(training_data)
    E_68,E_96 = confidence_interval(E_rbm_samples)
                                     
    print('r={} [], E(rbm)={} [], E(true)={} []'.format(rk,E_68[1],E_true))
    
    E_rbm_r_high_68.append(E_68[2])
    E_rbm_r_high_96.append(E_96[2])
                
    E_rbm_r_low_68.append(E_68[0])
    E_rbm_r_low_96.append(E_96[0])
    
    E_rbm_r_med.append(E_68[1])
    E_true_r.append(E_true)

r=0.2 [], E(rbm)=-1.0387497868145805 [], E(true)=-1.0272612255781348 []
r=0.45 [], E(rbm)=-1.1971074646979543 [], E(true)=-1.0362617443402724 []
r=0.7 [], E(rbm)=-1.2712140654698523 [], E(true)=-1.046658951711928 []
r=0.95 [], E(rbm)=-1.2384699693841665 [], E(true)=-1.05858094757703 []
r=1.2 [], E(rbm)=-1.1398168484798255 [], E(true)=-1.0642022250418146 []
r=1.45 [], E(rbm)=-0.995659564994946 [], E(true)=-1.0550336434246872 []
r=1.7 [], E(rbm)=-0.9225113460410881 [], E(true)=-1.0311415155381198 []
r=1.95 [], E(rbm)=-0.8623698106887224 [], E(true)=-0.9798311216602571 []
r=2.2 [], E(rbm)=-0.8270299177593836 [], E(true)=-0.9397402628548327 []
r=2.45 [], E(rbm)=-0.8135990400163524 [], E(true)=-0.9078290367824813 []
r=2.7 [], E(rbm)=-0.8011799296995397 [], E(true)=-0.8762670756779187 []


In [99]:
plt.figure(figsize=(15,5))
#plt.plot(E_rbm_r_low_68)
plt.plot(r_subset,E_rbm_r_med,label='E(RBM) median',color='blue')
#plt.plot(E_rbm_r_high_68)
plt.fill_between(r_subset,E_rbm_r_low_68, E_rbm_r_high_68,color='blue',alpha='0.4',edgecolor='b') 
plt.fill_between(r_subset,E_rbm_r_low_96, E_rbm_r_high_96,color='blue',alpha='0.2',edgecolor='b') 
plt.plot(r_subset,E_true_r,label='Exact energy',color='red')
plt.ylabel('Energy []',size=15)
plt.xlabel('Radius []',size=15)
plt.legend()
plt.show()